In [43]:
import requests, json, pandas as pd
pd.options.plotting.backend = "plotly"
from collections import defaultdict
import matplotlib.pyplot as plt
%matplotlib inline

In [54]:
statewise_uri = "https://api.covid19india.org/v3/timeseries.json"

def getTimeSeriesDataStateWise():
    req = requests.get(statewise_uri, verify=False)
    return req.json()

In [55]:
def createTableFromJson(jsondata):
    # col = ['date', 'state','delta_confirmed', 'delta_recovered', 'delta_tested',
    #        'total_confirmed', 'total_deceased', 'total_recovered', 'total_tested']

    d = defaultdict(list)

    for state, data in jsondata.items():
        for date, v in data.items():
            d['state'].append(state)
            d['date'].append(date)
            delta = v.get('delta', {})
            d['delta_confirmed'].append(delta.get('confirmed', 0))
            d['delta_deceased'].append(delta.get('deceased', 0))
            d['delta_recovered'].append(delta.get('recovered', 0))
            d['delta_tested'].append(delta.get('tested', 0))
            total = v.get('total', {})
            d['total_confirmed'].append(total.get('confirmed', 0))
            d['total_deceased'].append(total.get('deceased', 0))
            d['total_recovered'].append(total.get('recovered', 0))
            d['total_tested'].append(total.get('tested', 0))

    df = pd.DataFrame.from_dict(d)
    return df

In [62]:

def cleanData(df):
    mask_curr_date = (df['date'] > '2020-06-27') & (df['state'] != 'TT')
    new_df = df.loc[mask_curr_date].sort_values(by=['total_confirmed'], ascending=False)
    top_states = []
    for state in new_df[['state']].head(5).values.tolist():
        top_states.append(state[0])
    print(top_states)

    basic_mask = (df['date'] > '2020-04-15') & (df['state'] != 'TT') & (df['state'] != 'UN')
    top_states_mask, remaining_states_mask = False, basic_mask
    for state in top_states:
        top_states_mask = top_states_mask | (df['state'] == state)
        remaining_states_mask = remaining_states_mask & (df['state'] != state)

    top_states_mask = basic_mask & top_states_mask
    df['delta_confirmed_sma'] = df.iloc[:, 2].rolling(window=3).mean()
    top_state_df = pd.pivot_table(df.loc[top_states_mask], values='delta_confirmed_sma', index=['state'], 
                                  columns='date')
    rem_state_df = pd.pivot_table(df.loc[remaining_states_mask], values='delta_confirmed_sma', index=['state'], 
                                  columns='date')
    return (top_state_df, rem_state_df)

In [63]:
# with open('raw_data/timeseries.json') as file:
#     sj = json.load(file)
sj = getTimeSeriesDataStateWise()
df = createTableFromJson(sj)
df.head()

d:\learning\pyproj\viz_heatmap_covid\venv\lib\site-packages\urllib3\connectionpool.py:986: InsecureRequestWarning:

Unverified HTTPS request is being made to host 'api.covid19india.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings



,state,date,delta_confirmed,delta_deceased,delta_recovered,delta_tested,total_confirmed,total_deceased,total_recovered,total_tested
0,AN,2020-03-26,1,0,0,0,1,0,0,0
1,AN,2020-03-27,5,0,0,0,6,0,0,0
2,AN,2020-03-28,3,0,0,0,9,0,0,0
3,AN,2020-03-29,0,0,0,0,9,0,0,0
4,AN,2020-03-30,1,0,0,0,10,0,0,0


In [64]:
top_state_df, rem_state_df = cleanData(df)

['MH', 'DL', 'TN', 'GJ', 'UP']


In [65]:
top_state_df.head()

date,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-20,2020-04-21,2020-04-22,2020-04-23,2020-04-24,2020-04-25,...,2020-06-19,2020-06-20,2020-06-21,2020-06-22,2020-06-23,2020-06-24,2020-06-25,2020-06-26,2020-06-27,2020-06-28
state,,,,,,,,,,,,,,,,,,,,,
DL,43.333333,48.666667,105.000000,121.000000,124.666667,87.666667,81.666667,98.333333,119.333333,125.666667,...,2809.333333,3214.666667,3255.666667,3179.666667,3285.333333,3548.000000,3708.333333,3546.000000,3266.000000,3099.000000
GJ,119.000000,149.666667,203.333333,271.333333,280.000000,267.333333,221.333333,228.333333,212.333333,221.333333,...,523.333333,529.666667,553.000000,560.666667,564.000000,561.333333,566.000000,576.333333,590.666667,606.333333
MH,289.000000,213.666667,244.000000,333.000000,448.333333,523.333333,483.000000,587.000000,533.000000,659.666667,...,3628.666667,3817.666667,3857.000000,3821.666667,3601.666667,3608.000000,3981.666667,4585.000000,5411.333333,5628.333333
TN,31.333333,39.666667,43.333333,70.000000,65.666667,74.666667,50.666667,54.333333,53.000000,64.000000,...,2143.333333,2217.333333,2347.666667,2546.000000,2586.000000,2697.000000,2963.333333,3339.666667,3622.333333,3766.000000
UP,82.333333,63.000000,80.000000,98.333333,111.666667,120.666667,116.333333,108.666667,94.666667,114.666667,...,665.333333,651.333333,648.666667,576.000000,586.000000,608.666667,623.666667,683.333333,664.000000,651.333333


In [66]:
rem_state_df.head()

date,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-20,2020-04-21,2020-04-22,2020-04-23,2020-04-24,2020-04-25,...,2020-06-19,2020-06-20,2020-06-21,2020-06-22,2020-06-23,2020-06-24,2020-06-25,2020-06-26,2020-06-27,2020-06-28
state,,,,,,,,,,,,,,,,,,,,,
AN,0.000000,0.333333,1.000000,1.333333,1.333333,1.000000,1.000000,2.000000,4.000000,5.000000,...,1.000000,1.000000,1.000000,0.333333,1.000000,2.666667,3.333333,7.333333,5.333333,8.333333
AP,31.666667,29.333333,26.000000,37.666667,50.000000,51.333333,55.333333,57.000000,66.000000,67.666667,...,413.666667,460.333333,477.666667,470.333333,460.666667,467.333333,504.000000,551.666667,651.333333,738.000000
AR,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,12.000000,10.666667,12.000000,4.333333,7.666667,7.000000,8.000000,5.333333,5.666667,3.333333
AS,1.000000,0.666667,0.666667,0.000000,0.000000,0.000000,0.000000,0.333333,0.333333,0.333333,...,165.000000,186.666667,227.666667,282.666667,267.333333,232.000000,264.333333,287.666667,294.333333,282.000000
BR,5.666667,6.333333,4.666667,4.333333,9.333333,13.333333,15.666667,19.000000,32.333333,36.000000,...,160.000000,187.666667,208.333333,201.000000,182.333333,202.666667,198.333333,209.333333,235.666667,245.333333


In [76]:
fig = top_state_df.T.plot(kind = 'area',title="Cadence of cases for Top Indian States")
fig.show()

In [68]:
rem_state_df[rem_state_df < 0] = 0
fig = rem_state_df.T.plot.area(title="Cadence of cases for other Indian States")
fig.show()

In [69]:
def df_to_plotly(df):
    return {'z': df.values.tolist(),
            'x': df.columns.tolist(),
            'y': df.index.tolist()}

In [70]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Heatmap(df_to_plotly(top_state_df)))
fig.show()

In [72]:
fig = go.Figure(data=go.Heatmap(df_to_plotly(rem_state_df),colorscale='OrRd'))
fig.show()